In [3]:
import pandas as pd
import numpy as np
import joblib
import os

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [6]:
# Prepare data for battery health prediction

In [7]:
def prepare_battery_health_data(df):
    feature_cols = [
        'age_years', 'mileage', 'battery_cycles', 'fast_charge_ratio',
        'avg_temperature_c', 'avg_depth_of_discharge', 'voltage',
        'internal_resistance_mohm', 'current_capacity_kwh', 'charging_efficiency'
    ]
    
    le_model = LabelEncoder()
    df['model_encoded'] = le_model.fit_transform(df['model'])
    feature_cols.append('model_encoded')
    
    X = df[feature_cols].copy()
    y = df['battery_health_percent']
    
    return X, y, le_model, feature_cols

In [8]:
# Defind model

In [9]:
def train_battery_health_model(df):
    X, y, le_model, feature_cols = prepare_battery_health_data(df)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    model = GradientBoostingRegressor(
        n_estimators=200,
        max_depth=10,
        learning_rate=0.1,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    print("Battery Health Model Performance")
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
    print("R2:", r2_score(y_test, y_pred))
    
    joblib.dump(model, 'models/battery_health_model.pkl')
    joblib.dump(le_model, 'models/battery_health_label_encoder.pkl')
    joblib.dump(feature_cols, 'models/battery_health_features.pkl')
    
    return model


In [10]:
os.makedirs('models', exist_ok=True)

In [11]:
df = pd.read_csv('ev_battery_dataset.csv')
df.head()

,vehicle_id,model,age_years,age_months,mileage,battery_capacity_kwh,current_capacity_kwh,battery_health_percent,battery_cycles,voltage,...,avg_temperature_c,battery_temp_c,charging_efficiency,current_range_miles,current_soc_percent,avg_depth_of_discharge,base_price,needs_repair,repair_type,repair_cost
0,EV_00001,Volkswagen ID.4,1.41,16,11258,82,61.89,75.47,75,399.85,...,27.7,24.5,0.911,221.1,56.0,0.629,41000,True,Battery Cooling System,1204.87
1,EV_00002,Tesla Model 3,2.69,32,38103,75,37.50,50.00,254,387.33,...,14.0,4.2,0.876,151.6,58.3,0.819,50000,True,Battery Pack Reconditioning,11144.94
2,EV_00003,Chevrolet Bolt,0.61,7,8988,66,43.00,65.15,59,391.10,...,4.2,8.6,0.927,179.6,25.2,0.366,37000,True,Charging Port Repair,365.89
3,EV_00004,Nissan Leaf,1.33,15,19599,40,23.13,57.82,130,396.43,...,31.8,35.9,0.939,80.4,21.8,0.480,32000,True,Battery Management System,3291.99
4,EV_00005,Rivian R1T,0.34,4,4217,135,121.21,89.79,28,395.45,...,20.2,21.1,0.946,518.1,82.7,0.844,75000,False,No Repair Needed,0.00


In [12]:
# Train Model

In [13]:
battery_model = train_battery_health_model(df)

Battery Health Model Performance
MAE: 1.3207590201091786
RMSE: 2.4375768887071088
R2: 0.9651131616464996


In [14]:
# Test Model

In [15]:
# Load Saved Model & Encoders
model = joblib.load('models/battery_health_model.pkl')
le_model = joblib.load('models/battery_health_label_encoder.pkl')
feature_cols = joblib.load('models/battery_health_features.pkl')

In [17]:
# Create Test Input
sample_input = {
    'age_years': 3,
    'mileage': 45000,
    'battery_cycles': 800,
    'fast_charge_ratio': 0.35,
    'avg_temperature_c': 32,
    'avg_depth_of_discharge': 0.7,
    'voltage': 380,
    'internal_resistance_mohm': 4.5,
    'current_capacity_kwh': 52,
    'charging_efficiency': 0.92,
    'model': 'Tesla Model 3'
}

In [18]:
input_df = pd.DataFrame([sample_input])

In [19]:
input_df['model_encoded'] = le_model.transform(input_df['model'])
input_df.drop(columns=['model'], inplace=True)

In [20]:
input_df = input_df[feature_cols]

In [21]:
prediction = model.predict(input_df)

print(f"🔋 Predicted Battery Health: {prediction[0]:.2f}%")

🔋 Predicted Battery Health: 50.72%
